## Скачиваем необходимое

Сначала нужно средствами NLTK загрузить WordNet.



In [4]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

## Готовим данные к работе

Затем импортируем данные из подготовленного текстового файла. Файл содержит набор пар слов (только имён существительных), для которых известны экспертные оценки сходства.

Строим ассоциативный массив "пара слов -- оценка близости".

In [21]:
with open("wordsim_similarity_goldstandard.txt", encoding="utf-8") as rf:
  triples = [line.strip().split("\t") for line in rf.readlines()]
  score_map = {tuple(triple[:2]): float(triple[2]) for triple in triples}

In [ ]:
score_map

In [27]:
import csv
triples = []
score_map = {}
with open('Task_4_sample_4.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)  # skip the header
    for row in reader:
        pair = (row[0], row[1])
        score = float(row[2])
        triples.append(pair)
        score_map[pair] = score

In [ ]:
score_map

Отметим, что из исходного набора данных мы взяли только экспертные оценки сходства (similarity) и только для существительных. Исходный набор данных доступен по [ссылке](http://alfonseca.org/pubs/ws353simrel.tar.gz)

Посмотрим на примеры оценок. 

У слов может быть по несколько значений, которые различаются в WordNet. Здесь -- ради примера -- мы будем "жадно" выбирать первое попавшееся, но далее будем работать с ними иначе.




In [28]:
for w1, w2 in list(score_map)[:2]:
  
  print("\nWords: %s-%s\nGround truth score: %.2f" % (w1, w2, score_map[(w1, w2)]))
  
  ss1 = wn.synset(w1 + ".n.01")
  ss2 = wn.synset(w2 + ".n.01")

  print("\nPath: %.3f" % ss1.path_similarity(ss2), end=" ")
  print("\nwup: %.3f" % ss1.wup_similarity(ss2), end=" ")
  print("\nshortest_path: %.3f" % ss1.shortest_path_distance(ss2))


Words: professor-cucumber
Ground truth score: 0.31

Path: 0.077 
wup: 0.500 
shortest_path: 12.000

Words: monk-slave
Ground truth score: 0.92

Path: 0.200 
wup: 0.667 
shortest_path: 4.000


Вычисляем для всех пар несколько оценок

In [25]:
from itertools import product

list_pairs = list(score_map)
wup_list, true_list, path_list = [], [], []

# для всех пар
for w1, w2 in list_pairs:

  try:
    all_w1 = wn.synsets(w1, pos="n")
    all_w2 = wn.synsets(w2, pos="n")

    # добавляем интересующие нас метрики и экспертные оценки
    wup = max([item1.wup_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    wup_list.append(wup)

    path = max([item1.path_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    path_list.append(path)
    
    true_list.append(score_map[(w1, w2)])

  except Exception as e:
    print(w1, w2, "error:", e)

## Вычисляем ранговую корреляцию Спирмена

In [26]:
from scipy.stats import spearmanr

coef, p = spearmanr(wup_list, true_list)
print("wup  Spearman R: %.4f" % coef)

coef, p = spearmanr(path_list, true_list)
print("path Spearman R: %.4f" % coef)


wup  Spearman R: 0.6936
path Spearman R: 0.6535


In [29]:
# Calculate similarity scores and true scores for each pair
wup_list, true_list, path_list, lch_list = [], [], [], []
for w1, w2 in triples:
    try:
        all_w1 = wn.synsets(w1, pos='n')
        all_w2 = wn.synsets(w2, pos='n')

        # Compute similarity scores
        wup = max([item1.wup_similarity(item2) for item1, item2 in product(all_w1, all_w2)])
        wup_list.append(wup)

        path = max([item1.path_similarity(item2) for item1, item2 in product(all_w1, all_w2)])
        path_list.append(path)

        lch = max([item1.lch_similarity(item2) for item1, item2 in product(all_w1, all_w2)])
        lch_list.append(lch)

        # Add true scores
        true_list.append(score_map[(w1, w2)])
    except:
        continue

# Compute Spearman correlation coefficients for each similarity measure
wup_coef, _ = spearmanr(wup_list, true_list)
path_coef, _ = spearmanr(path_list, true_list)
lch_coef, _ = spearmanr(lch_list, true_list)

# Print the results
print('Spearman R for wup_similarity: {:.4f}'.format(wup_coef))
print('Spearman R for path_similarity: {:.4f}'.format(path_coef))
print('Spearman R for lch_similarity: {:.4f}'.format(lch_coef))

# Find the number of hyponyms for the synset "wood.n.01" and the name of the first hyponym
wood_synset = wn.synset('wood.n.01')
hyponyms = wood_synset.hyponyms()
num_hyponyms = len(hyponyms)
first_hyponym_name = hyponyms[0].name()

print('Number of hyponyms for wood.n.01:', num_hyponyms)
print('Name of the first hyponym:', first_hyponym_name)

Spearman R for wup_similarity: 0.6936
Spearman R for path_similarity: 0.6535
Spearman R for lch_similarity: 0.6535
Number of hyponyms for wood.n.01: 91
Name of the first hyponym: alder.n.01


wup  Spearman R: 0.6438
path Spearman R: 0.6176